# README  
This notebook mainly did two things:  
Given the 400 olfactory-related GPCRs, extract their mutation data from GPCRdb. - get in vitro data  
At the same time, run Ensembl VEPs on the mutation data. - get VEP scores

# preliminary 

In [2]:

import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import requests




In [3]:
# 20 types of amino acids
AA_all = {
    "A" : "Ala",
    "R" : "Arg",
    "N" : "Asn",
    "D" : "Asp",
    "C" : "Cys",
    "E" : "Glu",
    "Q" : "Gln",
    "G" : "Gly",
    "H" : "His",
    "I" : "Ile",
    "L" : "Leu",
    "K" : "Lys",
    "M" : "Met",
    "F" : "Phe",
    "P" : "Pro",
    "S" : "Ser",
    "T" : "Thr",
    "W" : "Trp",
    "Y" : "Tyr",
    "V" : "Val"
}

In [4]:
# read in all gene names
gene_list_raw =pd.read_csv("230323_EST_ENSG_GENE_new.csv")
# gene_list_raw
protein_list = gene_list_raw["protein"].tolist()
enst_list = gene_list_raw["ENST"].tolist()


In [5]:
gene_list_raw

,index,ENST,ENSG,protein,gene,Uniprot
0,0,ENST00000646641,ENSG00000267534,s1pr2_human,S1PR2,O95136
1,121,ENST00000547270,ENSG00000257138,t2r38_human,TAS2R38,P59533
2,219,ENST00000390675,ENSG00000256436,t2r31_human,TAS2R31,P59538
3,344,ENST00000539585,ENSG00000256188,t2r30_human,TAS2R30,P59541
4,483,ENST00000538986,ENSG00000255837,t2r20_human,TAS2R20,P59543
...,...,...,...,...,...,...
395,64029,ENST00000510937,ENSG00000226306,npy6r_human,NPY6R,Q99463
396,64030,ENST00000641193,ENSG00000279301,o2t11_human,OR2T11,Q8NH01
397,64031,ENST00000641732,ENSG00000172146,or1a1_human,OR1A1,Q9P1Q5
398,64032,ENST00000328890,ENSG00000183024,or1g1_human,OR1G1,P47890


In [ ]:
# gname = gene_list[124]
# # gname = 'o2t11_human'
# # enst = enst_list [0]
# url = 'https://gpcrdb.org/services/mutants/'+gname
# response = requests.get(url)
# mutants_data = response.json()
# df = pd.DataFrame(mutants_data)
# df

In [6]:
# check how many proteins have mutants info 
count = 0
mutant_genes =[]
for i in range(len(protein_list)):
    gname = protein_list[i]
    url = 'https://gpcrdb.org/services/mutants/'+gname
    response = requests.get(url)
    mutants_data = response.json()
    if(len(mutants_data)>0):
      count = count + 1
      mutant_genes.append(gname)
    else:
      print(gname+' info not found!')

print('there are '+str(len(mutant_genes))+' genes have mutants info')

s1pr2_human info not found!
t2r38_human info not found!
t2r31_human info not found!
t2r30_human info not found!
t2r20_human info not found!
hcar3_human info not found!
t2r43_human info not found!
taar9_human info not found!
t2r39_human info not found!
t2r46_human info not found!
t2r40_human info not found!
t2r41_human info not found!
gpr33_human info not found!
s1pr3_human info not found!
t2r13_human info not found!
t2r14_human info not found!
t2r19_human info not found!
lgr4_human info not found!
gpr20_human info not found!
npy4r_human info not found!
gpr52_human info not found!
sucr1_human info not found!
v1br_human info not found!
hcar1_human info not found!
gpr21_human info not found!
fpr3_human info not found!
gp141_human info not found!
ffar4_human info not found!
t2r42_human info not found!
ffar3_human info not found!
lpar5_human info not found!
ccr10_human info not found!
gp173_human info not found!
gpr39_human info not found!
ccr4_human info not found!
npbw1_human info not fou

there are 166 genes have mutants info

In [7]:
# only extract the info of those 167 genes
mutant_genes_df = gene_list_raw[gene_list_raw['protein'].isin(mutant_genes)]
mutant_genes_df.reset_index(drop=True, inplace=True)
mutant_genes_df

,index,ENST,ENSG,protein,gene,Uniprot
0,838,ENST00000321826,ENSG00000244165,p2y11_human,P2RY11,Q96G91
1,1668,ENST00000345363,ENSG00000213903,lt4r1_human,LTB4R,Q15722
2,2679,ENST00000377034,ENSG00000204681,gabr1_human,GABBR1,Q9UBS5
3,3455,ENST00000355085,ENSG00000197405,c5ar1_human,C5AR1,P21730
4,3758,ENST00000438284,ENSG00000196639,hrh1_human,HRH1,P35367
...,...,...,...,...,...,...
161,62409,ENST00000337539,ENSG00000128271,aa2ar_human,ADORA2A,P29274
162,62533,ENST00000620793,ENSG00000274286,ada2b_human,ADRA2B,P18089
163,62696,ENST00000610913,ENSG00000278195,ssr3_human,SSTR3,P32745
164,62881,ENST00000280155,ENSG00000150594,ada2a_human,ADRA2A,P08913


In [8]:
# in 166 genes, find if their enst have info in ensemble(because later we need to use enst id as input to ensembl vep)
server = "https://rest.ensembl.org"
for i in range(mutant_genes_df.shape[0]):
    gname = mutant_genes_df['gene'][i]
    enst = mutant_genes_df['ENST'][i]
    # ensg = mutant_genes_df['ENSG'][i]
    ext = "/sequence/id/"+enst+"?type=cdna"
    r = requests.get(server+ext, headers={ "Content-Type" : "text/x-fasta"})
    if not r.ok:
      print(gname+' ENST not OK')

# Get access to GPCRdb

all ENSTs are OK! Now next extract mutation data. create two dataframes: gpcr_all_mutants stores all data from GPCRsb, vep_input all stores the mutation info(but transform from 1 letter abbreviation to 3 letter abbreviation) as the further inputs to Ensembl VEP

In [16]:
gpcr_all_mutants = pd.DataFrame()
vep_input_all = pd.DataFrame()
for i in range(mutant_genes_df.shape[0]):
  gname = mutant_genes_df['protein'][i]
  enst = mutant_genes_df['ENST'][i]
  url = 'https://gpcrdb.org/services/mutants/'+gname
  response = requests.get(url)
  mutants_data = response.json()
  df = pd.DataFrame(mutants_data)
  # only keep missense mutation
  df = df[df['mutation_to']!='-']
  df = df[df['mutation_to']!='X']
  df = df[df['mutation_to']!='']
  df = df[df['mutation_from']!=df['mutation_to']]
  df.reset_index(drop=True, inplace=True)
  gpcr_all_mutants = gpcr_all_mutants.append(df)
  vep_input_df = pd.DataFrame()
  print("processing No."+str(i)+" ---> "+gname)
  for j in range(df.shape[0]):
    row = vep_input_df.shape[0]
    vep_input_df.loc[row,'protein'] = gname
    vep_input_df.loc[row,'ENST'] = enst
    vep_input_df.loc[row,'mutation_pos'] = df.loc[j,"mutation_pos"]
    # we need to store this column so that we can filter out rows with no exp_fold_change value
    vep_input_df.loc[row,'exp_fold_change'] = df.loc[j,"exp_fold_change"]
    vep_input_df.loc[row,'mutation_from'] = AA_all[df.loc[j,"mutation_from"]] 
    vep_input_df.loc[row,'mutation_to'] = AA_all[df.loc[j,"mutation_to"]]
  vep_input_df['mutation_pos'] = vep_input_df['mutation_pos'].astype(int) 
  vep_input_df['vep_input'] = enst + ':p.' + vep_input_df['mutation_from'] + vep_input_df['mutation_pos'].astype(str) + vep_input_df['mutation_to']
  
  vep_input_all = vep_input_all.append(vep_input_df)



processing No.0 ---> p2y11_human
processing No.1 ---> lt4r1_human
processing No.2 ---> gabr1_human
processing No.3 ---> c5ar1_human
processing No.4 ---> hrh1_human
processing No.5 ---> cnr2_human
processing No.6 ---> adrb3_human
processing No.7 ---> fzd9_human
processing No.8 ---> p2ry4_human
processing No.9 ---> t2r60_human
processing No.10 ---> acthr_human
processing No.11 ---> npy2r_human
processing No.12 ---> acm5_human
processing No.13 ---> drd1_human
processing No.14 ---> rl3r1_human
processing No.15 ---> acm2_human
processing No.16 ---> oxyr_human
processing No.17 ---> cxcr2_human
processing No.18 ---> agtr2_human
processing No.19 ---> acm4_human
processing No.20 ---> 5ht1d_human
processing No.21 ---> 5ht1f_human
processing No.22 ---> 5ht1a_human
processing No.23 ---> mc5r_human
processing No.24 ---> p2ry2_human
processing No.25 ---> trfr_human
processing No.26 ---> agrg2_human
processing No.27 ---> cltr1_human
processing No.28 ---> ada1d_human
processing No.29 ---> c3ar_human
p

In [17]:
print(vep_input_all.shape)
print(gpcr_all_mutants.shape)

(24455, 7)
(24455, 25)


In [11]:
# we need to delete row with exp_fold_change==0 for both vep_input_all and gpcr_all_mutants
# drop the lines with exp_fold_change==0 
gpcr_filtered_mutants = gpcr_all_mutants[gpcr_all_mutants['exp_fold_change']!=0]
gpcr_filtered_mutants

,reference,protein,mutation_pos,mutation_from,mutation_to,ligand_name,ligand_idtype,ligand_id,ligand_class,exp_type,...,exp_mu_effect_value,exp_fold_change,exp_mu_effect_qual,exp_mu_effect_ligand_prop,exp_mu_ligand_ref,opt_receptor_expression,opt_basal_activity,opt_gain_of_activity,opt_ligand_emax,opt_agonist
22,15626750,hrh1_human,37,I,V,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,pK(i),...,7.700,-1.585,,,[3H]-mepyramine (radioligand),94.2,0,,0,
23,15626750,hrh1_human,38,C,S,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,pK(i),...,7.900,1.000,,,[3H]-mepyramine (radioligand),141.3,0,,0,
24,15626750,hrh1_human,84,N,S,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,pK(i),...,7.500,-2.513,,,[3H]-mepyramine (radioligand),53.7,0,,0,
26,16408006,hrh1_human,111,S,A,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,pK(i),...,7.300,1.000,,,[3H]-mepyramine (radioligand),58.3,0,,0,
27,16408006,hrh1_human,111,S,C,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,pK(i),...,7.500,1.585,,,[3H]-mepyramine (radioligand),55.0,0,,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,10.1124/mol.63.5.1021,aa3r_human,274,N,A,VUF-5455,PubChem CID,10339071,Allosteric inverse agonist,k-1,...,0.083,-3.322,,,I-AB-MECA,0,0,=,0,
456,10.1124/mol.63.5.1021,aa3r_human,274,N,A,I-AB-MECA,PubChem CID,44208896,Full agonist,k-1,...,0.085,-1.328,,,,0,0,=,0,
457,10.1006/bbrc.2001.5027,aa3r_human,282,Y,F,Cl-IB-MECA,PubChem CID,393593,Full agonist,EC50,...,27.600,-29.412,,,,0,0,=,0,
458,10.1006/bbrc.2001.5027,aa3r_human,282,Y,F,I-AB-MECA,PubChem CID,44208896,Full agonist,K(d),...,9.300,-7.752,,,,0,0,=,0,


In [12]:
unq_gpcr = gpcr_filtered_mutants.drop_duplicates(subset=['protein','mutation_pos','mutation_from','mutation_to'], keep='first', inplace=False)
unq_gpcr

,reference,protein,mutation_pos,mutation_from,mutation_to,ligand_name,ligand_idtype,ligand_id,ligand_class,exp_type,...,exp_mu_effect_value,exp_fold_change,exp_mu_effect_qual,exp_mu_effect_ligand_prop,exp_mu_ligand_ref,opt_receptor_expression,opt_basal_activity,opt_gain_of_activity,opt_ligand_emax,opt_agonist
22,15626750,hrh1_human,37,I,V,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,pK(i),...,7.70,-1.585,,,[3H]-mepyramine (radioligand),94.2,0,,0,
23,15626750,hrh1_human,38,C,S,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,pK(i),...,7.90,1.000,,,[3H]-mepyramine (radioligand),141.3,0,,0,
24,15626750,hrh1_human,84,N,S,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,pK(i),...,7.50,-2.513,,,[3H]-mepyramine (radioligand),53.7,0,,0,
26,16408006,hrh1_human,111,S,A,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,pK(i),...,7.30,1.000,,,[3H]-mepyramine (radioligand),58.3,0,,0,
27,16408006,hrh1_human,111,S,C,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,pK(i),...,7.50,1.585,,,[3H]-mepyramine (radioligand),55.0,0,,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,10.1074/jbc.M110960200,aa3r_human,247,S,A,I-AB-MECA,PubChem CID,44208896,Full agonist,K(d),...,1.70,1.000,,,,0,0,=,0,
391,10.1074/jbc.M110960200,aa3r_human,272,H,E,CADO,PubChem CID,8974,Full agonist,K(i),...,11.60,-22.222,,,I-AB-MECA,0,0,=,0,
433,10.1021/jm050968b,aa3r_human,272,H,D,CID75022615,PubChem CID,75022615,Full agonist,K(i),...,0.12,1.167,,,I-AB-MECA,0,0,=,0,
446,10.1124/mol.63.5.1021,aa3r_human,274,N,A,I-AB-MECA,PubChem CID,44208896,Full agonist,K(d),...,4.60,-3.831,,,,0,0,=,0,


In [18]:
vep_filtered_all = vep_input_all[vep_input_all['exp_fold_change']!=0]
vep_filtered_all

,protein,ENST,mutation_pos,exp_fold_change,mutation_from,mutation_to,vep_input
22,hrh1_human,ENST00000438284,37,-1.585,Ile,Val,ENST00000438284:p.Ile37Val
23,hrh1_human,ENST00000438284,38,1.000,Cys,Ser,ENST00000438284:p.Cys38Ser
24,hrh1_human,ENST00000438284,84,-2.513,Asn,Ser,ENST00000438284:p.Asn84Ser
26,hrh1_human,ENST00000438284,111,1.000,Ser,Ala,ENST00000438284:p.Ser111Ala
27,hrh1_human,ENST00000438284,111,1.585,Ser,Cys,ENST00000438284:p.Ser111Cys
...,...,...,...,...,...,...,...
455,aa3r_human,ENST00000241356,274,-3.322,Asn,Ala,ENST00000241356:p.Asn274Ala
456,aa3r_human,ENST00000241356,274,-1.328,Asn,Ala,ENST00000241356:p.Asn274Ala
457,aa3r_human,ENST00000241356,282,-29.412,Tyr,Phe,ENST00000241356:p.Tyr282Phe
458,aa3r_human,ENST00000241356,282,-7.752,Tyr,Phe,ENST00000241356:p.Tyr282Phe


In [14]:
ll_vep = pd.DataFrame(np.unique(vep_filtered_all['vep_input'].tolist()))
#drop 'nan
# ll_vep = ll_vep[ll_vep[0]!='nan']
ll_vep

,0
0,ENST00000019103:p.Ala210Cys
1,ENST00000019103:p.Ala213Cys
2,ENST00000019103:p.Ala290Cys
3,ENST00000019103:p.Ala292Cys
4,ENST00000019103:p.Ala359Cys
...,...
3066,ENST00000646607:p.Lys273Ala
3067,ENST00000646607:p.Trp336Ala
3068,ENST00000646607:p.Tyr247Ala
3069,ENST00000646607:p.Tyr350Ala


# output data

In [19]:
vep_filtered_all.to_csv('vep_filtered_all.csv',index=False)

In [18]:
# save the filtered vep inputs
ll_vep.to_csv('all_vep_inputs.csv',index=False, header=False)


In [19]:
# save to csv
gpcr_filtered_mutants.to_csv("gpcr_filtered_mutants_0217.csv")